In [ ]:
!pip install tensorflow==2.11.0 opencv-python mediapipe sklearn matplotlib
import cv2
import mediapipe as mp
import numpy as np
import os
import tensorflow as tf
from google.colab.patches import cv2_imshow

### Preprocess data, add labels

In [ ]:
actions = np.array(['fidgety','cramped_synchronized'])

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
#create a dictionary
label_map = {label:num for num, label in enumerate(actions)}

In [ ]:
DATA_PATH = '/content/drive/MyDrive/Keypoints data/MP_data'
actions = np.array(['cramped_synchronized','fidgety'])

In [ ]:
sequence_length = 120

In [ ]:
sequences, labels = [], []
no_sequences = 30
action = 'cramped_synchronized'
for sequence in range(no_sequences):
  window =[]
  for frame_num in range (sequence_length):
    res = np.load(os.path.join(DATA_PATH, action, str(sequence), str(frame_num) + '.npy'))
    window.append(res)
  sequences.append(window)
  labels.append(label_map[action])

In [ ]:
no_sequences = 30
action = 'fidgety'
for sequence in range(no_sequences):
  window =[]
  for frame_num in range (sequence_length):
    res = np.load(os.path.join(DATA_PATH, action, str(sequence), str(frame_num) + '.npy'))
    window.append(res)
  sequences.append(window)
  labels.append(label_map[action])

In [ ]:
#/content/drive/MyDrive/Keypoints data/MP_data/cramped_synchronized
#/content/drive/MyDrive/Keypoints data/MP_data/fidgety

In [ ]:
np.array(sequences).shape


In [ ]:
X = np.array(sequences)

In [ ]:
X.shape


In [ ]:
y = to_categorical(labels).astype(int)

In [ ]:
y

In [ ]:
np.array(labels).shape

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size =0.05)

In [ ]:
y_test.shape

### Build and train LSTM Neural Network



In [ ]:
y = to_categorical(labels).astype(int)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size =0.1)

In [ ]:
X = np.load('/content/sample_data/X.npy')
X.shape

(106, 120, 132)

In [ ]:
labels = np.load('/content/sample_data/labels.npy')
labels = labels.tolist()
labels

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size =0.05)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
#create log directory and set up tensorflow callback
log_dir = os.path.join('Logs')
tb_callback = TensorBoard (log_dir = log_dir)

In [ ]:
model = Sequential()
model.add(LSTM(64,return_sequences=True, activation='relu', input_shape=(120,132)))
model.add(LSTM(128,return_sequences=True, activation='relu')) #true because there is another layer next
model.add(LSTM(64,return_sequences=False, activation='relu')) # false because this is the last layer
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [ ]:
model.compile(optimizer='Adam', loss = 'categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
#train the model
model.fit(x_train,y_train, epochs = 120, callbacks=[tb_callback])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 120, 64)           50432     
                                                                 
 lstm_4 (LSTM)               (None, 120, 128)          98816     
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 2)                 66        
                                                                 
Total params: 204,962
Trainable params: 204,962
Non-tr

In [ ]:
res = model.predict(x_test)

1/1 [==============================] - 0s 484ms/step


In [ ]:
actions[np.argmax(res[4])]

'cramped_synchronized'

In [ ]:
actions[np.argmax(y_test[5])]


'fidgety'

In [ ]:
model.predict(x_test)

1/1 [==============================] - 0s 57ms/step


array([[1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [ ]:
y_test

array([[0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [0, 1]])